迭代器

一个实现了__ iter __ 方法的是可迭代的（Iterable），而实现了方法__ next __ 的对象是迭代器（Iterator）

可迭代对象
一类是集合数据类型 如 list, tuple, dict, set, str等

一类是generator，包括生成器和带yield的生成器函数

这些可以直接作用于for循环的对象统称为可迭代对象：Iterable

实现了 __ iter __ 方法的对象是可迭代的

可以使用isinstance()判断一个对象是否是 Iterable （可迭代的）


In [9]:
from collections import Iterable

isinstance([], Iterable)

True

In [10]:
isinstance({}, Iterable)

True

In [11]:
isinstance('abc', Iterable)

True

In [12]:
isinstance((x for x in range(10)), Iterable)

True

In [13]:
isinstance(100, Iterable)

False

生成器不仅可以作用于for循环，还可以被 __ next __ () 函数调用并不断返回下一个值的对象。直到最后抛出StopIteration错误表示无法继续返回下一个值了。

可以被next()函数调用并不断返回下一个值的对象称为迭代器：Iterator

所以一个实现了 iter方法和next方法的对象就是迭代器。也可以说实现了方法 __ next __ 的可迭代对象时迭代器。

可以使用isinstance()判断一个对象是否是Iterator对象

In [24]:
from collections import Iterator

isinstance((x for x in range(10)), Iterator)

True

In [16]:
isinstance([], Iterator)

False

In [17]:
isinstance({}, Iterator)

False

In [18]:
isinstance('abc', Iterator)

False

总结：

凡是作用于for循环的都是可迭代的，实现了__ next __ 方法的 都是 迭代器。

生成器都是Iterator对象，但list，dict，str虽然是Iterable （可迭代的），却不是Iterator（迭代器）。

但是list,dict, str等Iterable 可以变成 Iterator 使用__ iter __ () 或 iter()

In [22]:
isinstance([].__iter__(),Iterator)

True

In [23]:
isinstance(iter([]), Iterator)

True

### for循环本质

我们都知道，迭代器对象，实现了__ next __ ()方法，可以不断调用该方法，返回迭代器对象的每个元素    

for循环的对象，必须是可迭代对象。而在底层，所有的可迭代对象均内置了__ iter __ ()方法，返回值为迭代器对象。

此时，我们便明了了，for循环本质上调用了可迭代对象的__ iter __ ()方法，得到了该对象对应的迭代器对象，然后无限调用__ next __ ()方法，得到对象中的每一个元素。直到StopIteration异常，并不会抛出这个异常。代表迭代器中已无下一个元素，for循环自动处理该异常，跳出循环。

In [7]:
x = [1, 2, 3]
x_i  =x.__iter__()
try:
    while True:
        print(x_i.__next__())
except StopIteration:
        pass

1
2
3


In [8]:
# 注意x_i已经是 迭代器对象
x_i

In [5]:
x = [1, 2, 3]
x.__next__()

AttributeError: 'list' object has no attribute '__next__'

In [7]:
x = x.__iter__()
x.__next__()

2

### 自定义迭代器对象

In [38]:
# 只定义 __ iter __ () 没定义 __ next __()
class MyRange:
    def __init__(self, num):
        self.i = 0
        self.num = num
    
    def __iter__(self):
        return self

In [37]:
isinstance(MyRange(4), Iterable)

True

In [39]:
isinstance(MyRange(4), Iterator)

False

可见：只实现了 __ iter __ () 方法 是可迭代的 但不是 迭代器 因为没实现 __ next __ () 方法

In [42]:
for i in MyRange(3):
    print(i)

TypeError: iter() returned non-iterator of type 'MyRange'

### for循环步骤：

1. 先判断对象是否是可迭代对象，不是的话直接报错，抛出TypeError异常，是的话调用__ iter __ ()方法，返回迭代器（只有实现了 next 方法才是迭代器）。如果没实现 __ next __ () 方法，报错 iter()返回的类型不是迭代器。所以说可以转化成迭代器的可迭代对象，才可以被for循环执行

2. 不断地调用迭代器的 __ next __ ()方法，每次按序返回迭代器中的一个值。

3. 迭代到最后，没有更多的元素了，就抛出StopIteration异常，这个异常Python自己处理，不暴露给开发者。

In [29]:
# 修改 MyRange
class MyRange:
    def __init__(self, num):
        self.i = 0
        self.num = num
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.i < self.num:
            i = self.i
            self.i += 1
            return i
        else:
            raise StopIteration()
#因为现在已经实现了 __ next __ ()方法，所以已经是迭代器。
for wew in MyRange(4):
    print(wew)
#for循环本质就是不断调用迭代器的 __ next __ () 方法，直到有 StopIteration 
#异常为止。

0
1
2
3


In [34]:
isinstance(MyRange(4), Iterable)

True

参考：
1. [Python for循环本质](https://www.cnblogs.com/linshuhui/p/8986094.html)

2. [for循环在Python中是怎么工作的](https://foofish.net/how-for-works-in-python.html)
3. [python 生成器和迭代器有这篇就够了](https://www.cnblogs.com/wj-1314/p/8490822.html)
4. [Python可迭代对象，迭代器，生成器的区别](https://blog.csdn.net/liangjisheng/article/details/7977600A8)

In [32]:
isinstance([], Iterable)

True

生成器

send()和next()一样，都能让生成器继续往下一步走。不同的是send传递值，next不传递值。

注意：
不能把send放第一个，因为send只能将值传给上一个yield所在的位置。

第一次执行程序是从开始执行。

并没有值来接受send。如果非要把send放第一个，那么传递的值应该是None。

In [1]:
def test():
    i = 0
    while i<5:
        temp = yield i
        print(temp)
        i += 1

In [2]:
a = test()

In [3]:
a.__next__()

0

下边返回None 是因为 第二次取值 直接从yield下一条语句执行，temp没有赋到值

In [4]:
a.__next__()

None


1

这说明 send 传递的值（hello）给了上一个yield所在的位置

In [5]:
a.send('hello')

hello


2

In [59]:
def test():
    i = 0 
    while i < 5:
        temp = yield i
        print(temp)
        i += 1

这说明 不能 在首次就用send传值 因为前面没有yield了 并没有值来接受yield

In [60]:
a = test()
a.send('fds')

TypeError: can't send non-None value to a just-started generator

但是可以给send()里边传None

In [61]:
a.send(None)

0

补充 判断可迭代对象

自定义对象再实现了iter()方法或者 **实现了** getitem()方法后 也是可迭代的

通过isinstance(obj, Iterable)判断一个对象是否是可迭代对象时，只有当这个对象被注册为Iterable或者当他实现了iter()方法时，才返回True。

而对于实现了getitem()方法的，返回的是False。所以稳妥判断是否是可迭代对象是调用iter(obj)，如果不报错，证明是可迭代对象。

同理。自定义对象转换迭代器，必须实现 iter 和 next 。同时实现 getitem 和 next 也是可以达到 next() 访问值的效果，但是通过 isinstance 判断返回 False，这里返回 False，应该和可迭代器判断返回 False 的原因是一致的
